
# MALARIA

In [ ]:
import pandas as pd
import seaborn as sp
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import KFold
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC

In [ ]:
malaria = pd.read_csv('Dataset/malaria_dataset.csv')
malaria.head()

In [ ]:
malaria.drop(['Unnamed: 0'], axis=1)


In [ ]:
malaria.drop_duplicates()

## DATA CLEANING

In [ ]:
malaria['DISTRICT'].fillna(malaria['DISTRICT'].mode()[0], inplace = True)
malaria['AGE'].fillna(malaria['AGE'].median(), inplace = True)
malaria['REPORT_VERIFIED'].fillna(malaria['REPORT_VERIFIED'].mode()[0], inplace = True)
malaria['TEHSIL'].fillna(malaria['TEHSIL'].mode()[0], inplace = True)

## DATA BALANCING

In [ ]:
#Class Distribution
class_counts_typ = malaria['RESULT_TEXT'].value_counts()

class_distribution_typ =class_counts_typ / len(malaria) *100

print(class_distribution_typ)

#plot a bar graph
value = malaria['RESULT_TEXT'].value_counts()

plt.bar(value.index, value.values)

plt.title('RESULT_TEXT')
plt.xlabel('Result')
plt.ylabel('Count')

plt.show()

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Separate features and target variable
x = malaria.drop('RESULT_TEXT', axis=1)
y = malaria['RESULT_TEXT']

# Undersample the majority class using RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
x_resampled, y_resampled = rus.fit_resample(x, y)

# Concatenate the features and target into balanced dataset
balanced_data = pd.concat([x_resampled, y_resampled], axis=1)

balanced_data.to_csv('new_copy2/Balanced_Malaria.csv', index = False)

Balanced_malaria = pd.read_csv('new_copy2/Balanced_Malaria.csv')

#Class Distribution
class_counts_typ = Balanced_malaria['RESULT_TEXT'].value_counts()

class_distribution_typ =class_counts_typ / len(Balanced_malaria) *100

print(class_distribution_typ)

#plot a bar graph
value = Balanced_malaria['RESULT_TEXT'].value_counts()

plt.bar(value.index, value.values)

plt.title('RESULT_TEXT')
plt.xlabel('Result')
plt.ylabel('Count')

plt.show()

In [ ]:
Balanced_malaria.info()

## DATA TRANSFORMATION

In [ ]:
le = LabelEncoder()
Balanced_malaria["MRNO_encoded"] = le.fit_transform(Balanced_malaria["MRNO"])
Balanced_malaria["RESULT_VALUE_encoded"] = le.fit_transform(Balanced_malaria["RESULT_VALUE"])
Balanced_malaria["GENDER_encoded"] = le.fit_transform(Balanced_malaria["GENDER"])
Balanced_malaria["REPORT_VERIFIED_encoded"] = le.fit_transform(Balanced_malaria["REPORT_VERIFIED"])
Balanced_malaria["RESULT_TEXT_encoded"] = le.fit_transform(Balanced_malaria["RESULT_TEXT"])
# One-hot encode District and Tehsil
ohe = OneHotEncoder(sparse=False)
district_tehsil_encoded = ohe.fit_transform(Balanced_malaria[["DISTRICT", "TEHSIL"]])
district_tehsil_encoded_df = pd.DataFrame(district_tehsil_encoded, columns=ohe.get_feature_names_out(["DISTRICT", "TEHSIL"]))

# Combine the encoded columns with the original dataset
new_df = pd.concat([Balanced_malaria["MRNO_encoded"], district_tehsil_encoded_df], axis=1)
new_df["AGE"] = Balanced_malaria["AGE"]
new_df["RESULT_TEXT"] = Balanced_malaria["RESULT_TEXT_encoded"]
new_df["GENDER"] = Balanced_malaria["GENDER_encoded"]
new_df["RESULT_VALUE"] = Balanced_malaria["RESULT_VALUE_encoded"]
new_df["REPORT_VERIFIED"] = Balanced_malaria["REPORT_VERIFIED_encoded"]
new_df["CPT_ID"] = Balanced_malaria["CPT_ID"]
new_df["CPT_ID.1"] = Balanced_malaria["CPT_ID.1"]
# Save the new dataframe to a new CSV file
new_df.to_csv('new_copy2/New_Malaria.csv', index = False)

In [ ]:
print(Balanced_malaria['MRNO_encoded'])

## DATA NORMALISATION

In [ ]:
value = Balanced_malaria['AGE'].value_counts()

plt.bar(value.index, value.values)

plt.title('AGE')
plt.xlabel('Age')
plt.ylabel('Count')

plt.show()

In [ ]:
Transformed_malaria = pd.read_csv('new_copy2/New_Malaria.csv', low_memory = False)
# Column to be normalized
column = ['AGE']

Transformed_malaria[column] = (Transformed_malaria[column] - Transformed_malaria[column].mean()) / Transformed_malaria[column].std()

# New .csv file with normalized data
Transformed_malaria.to_csv('new_copy2/Normalized_Malaria.csv', index = False)# Column to be normalized
column = ['AGE']

Transformed_malaria[column] = (Transformed_malaria[column] - Transformed_malaria[column].mean()) / Transformed_malaria[column].std()

# New .csv file with normalized data
Transformed_malaria.to_csv('new_copy2/Normalized_Malaria.csv', index = False)

In [ ]:
Normal_malaria = pd.read_csv('new_copy2/Normalized_Malaria.csv', low_memory = False)
print(Normal_malaria['AGE'].head())

value = Normal_malaria['AGE'].value_counts()

plt.bar(value.index, value.values)

plt.title('AGE')
plt.xlabel('Age')
plt.ylabel('Count')

plt.show()

## RANDOM FOREST FEATURE SELECTION

In [ ]:
# Drop rows with missing data
Normal_malaria.dropna(inplace=True)

# Split the data into feature matrix X and target vector y
X = Normal_malaria.drop('RESULT_TEXT', axis=1)
y = Normal_malaria['RESULT_TEXT']

In [ ]:
# Use random forest to select the most important features
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
importance = rfc.feature_importances_

In [ ]:
# Create a list of (feature name, importance) tuples and sort by importance
features = list(zip(X.columns, importance))
features.sort(key=lambda x: x[1], reverse=True)

# Print the sorted list of feature importances
for f in features:
    print(f)


# Select the top k features
k = 7
top_features = [f[0] for f in features[:k]]
for f in top_features:
    print(f)
X = X[top_features]

In [ ]:
plt.title("Histogram for Feature selection importance");
plt.barh([x[0] for x in top_features],[x[1] for x in top_features])
plt.show()

## RANDOM FOREST CLASSIFIER

In [ ]:
# Train and evaluate the model using KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
accuracy_scores = []
y_preds=[]
y_tests=[]
for train_index, test_index in kf.split(X):
    # Split data into train and test sets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Create and train the random forest classifier
    rfc = RandomForestClassifier(n_estimators=100)
    rfc.fit(X_train, y_train)

    # Make predictions on the test set and calculate accuracy
    y_pred = rfc.predict(X_test)
    y_preds.extend(y_pred)
    y_tests.extend(y_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Add accuracy score to list
    accuracy_scores.append(accuracy)

In [ ]:
# Compute and print the mean accuracy score and standard deviation
print("Accuracy- Random forest classifier: %0.2f (+/- %0.2f)" % (np.mean(accuracy_scores), np.std(accuracy_scores) * 2))

In [ ]:
# Assuming the true and predicted labels are stored in y_true and y_pred respectively
cm = confusion_matrix(y_tests, y_preds)

# Create a heatmap of the confusion matrix using Seaborn
sp.heatmap(cm, annot=True, cmap="Blues", fmt = 'd')

# Add axis labels and a title
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix for Random forest")
plt.show()

In [ ]:
# y_true: true labels, y_pred_prob: predicted probabilities
fpr, tpr, thresholds = roc_curve(y_tests, y_preds)

# plot ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')  # plot random curve
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Random Forest')
plt.show()

## KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

kfold = KFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []
# Perform K-fold cross-validation and evaluate the model's performance
y_true = []
y_pred = []
for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    knn.fit(X_train, y_train)
    y_true.extend(y_test)
    y_pred.extend(knn.predict(X_test))

In [ ]:
accuracy = accuracy_score(y_true, y_pred)
# Add accuracy score to list
accuracy_scores.append(accuracy)
# Compute and print the mean accuracy score and standard deviation
print("Accuracy- KNN: %0.2f (+/- %0.2f)" % (np.mean(accuracy_scores), np.std(accuracy_scores) * 2))

In [ ]:
# Assuming the true and predicted labels are stored in y_true and y_pred respectively
cm = confusion_matrix(y_true, y_pred)
# Create a heatmap of the confusion matrix using Seaborn
sp.heatmap(cm, annot=True, cmap="Blues", fmt="d")

# Add axis labels and a title
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix for KNN")

# Show the plot
plt.show()

In [ ]:
# y_true: true labels, y_pred_prob: predicted probabilities
fpr, tpr, thresholds = roc_curve(y_true, y_pred)

# plot ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')  # plot random curve
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for KNN')
plt.show()

In [ ]:
# y_true: true labels, y_pred_prob: predicted probabilities
fpr, tpr, thresholds = roc_curve(y_true, y_pred)

# plot ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')  # plot random curve
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for KNN')
plt.show()

## DECISION TREE

In [ ]:
# Initialize an empty list to store cross-validation scores
scores = []
y_pred=[]
y_preds=[]
y_tests=[]
# Iterate over the splits of the data and train/test the model
for train_index, test_index in kf.split(X):
    # Split the data into training and testing sets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Train the decision tree classifier
    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train, y_train)
    
    # Evaluate the model's performance on the test set for this fold and store the score
    y_pred = dt.predict(X_test)
    y_preds.extend(y_pred)
    y_tests.extend(y_test)
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

# Calculate the mean and standard deviation of the cross-validation scores
mean_score = sum(scores) / len(scores)
std_dev = np.std(scores)

# Print the results
print("Accuracy-Decision tree: %0.2f (+/- %0.2f)" % (mean_score, std_dev * 2))

In [ ]:
# Assuming the true and predicted labels are stored in y_true and y_pred respectively
cm = confusion_matrix(y_tests, y_preds)
np.set_printoptions(precision=3, suppress=True)
# Create a heatmap of the confusion matrix using Seaborn
sp.heatmap(cm, annot=True, cmap="Blues",fmt="d")

# Add axis labels and a title
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix for Decision tree")

# Show the plot
plt.show()

In [ ]:
# y_true: true labels, y_pred_prob: predicted probabilities
fpr, tpr, thresholds = roc_curve(y_tests, y_preds)

# plot ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')  # plot random curve
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Decision tree')
plt.show()

## Logistic Regression

In [ ]:
# Perform 10-fold cross validation using KFold method
kf = KFold(n_splits=10, shuffle=True, random_state=42)
lr = LogisticRegression(C=1)
scores = []
y_preds = []
y_true = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    y_preds.extend(y_pred)
    y_true.extend(y_test)
    scores.append(lr.score(X_test, y_test))

# Calculate and print the cross-validation accuracy
print("Accuracy- Logistic Regression: %0.2f (+/- %0.2f)" % (np.mean(scores), np.std(scores) * 2))

In [ ]:
import pandas as pd
from imblearn.combine import SMOTEENN
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix


# Define the hybrid sampling method
smote_enn = SMOTEENN(random_state=42)
scores = []
# Define the logistic regression classifier
logreg = LogisticRegression(random_state=42)

# Define the KFold cross-validator
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform K-fold cross-validation and evaluate the model's performance
for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Apply the hybrid sampling method on the training set only
    X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
    
    # Fit the logistic regression model on the resampled training set
    logreg.fit(X_resampled, y_resampled)
    
    # Make predictions on the test set
    y_pred = logreg.predict(X_test)
    
    # Evaluate the model's performance on the test set
    y_preds.extend(y_pred)
    y_true.extend(y_test)
    scores.append(lr.score(X_test, y_test))
    
    # Calculate the cross-validation accuracy
    #scores = cross_val_score(logreg, X_resampled, y_resampled, cv=kfold)

print("Cross-validation accuracy: %0.2f (+/- %0.2f)\n" % (np.mean(scores), np.std(scores) * 2))

In [ ]:
# Assuming the true and predicted labels are stored in y_true and y_pred respectively
cm = confusion_matrix(y_true, y_preds)
np.set_printoptions(precision=3, suppress=True)
# Create a heatmap of the confusion matrix using Seaborn
sp.heatmap(cm, annot=True, cmap="Blues",fmt="d")

# Add axis labels and a title
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix for Logistic Regression")

# Show the plot
plt.show()

In [ ]:
# y_true: true labels, y_pred_prob: predicted probabilities
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# plot ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')  # plot random curve
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Logisitic Regression')
plt.show()

## SVM

In [ ]:
y_pred=[]
y_preds=[]
y_true=[]
# Define the desired sample size for the reduced dataset
sample_size = 5000

# Initialize SVM classifier with default hyperparameters
svm = SVC()

# Use stratified k-fold cross-validation to evaluate classifier performance
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
scores = []

# Loop over each fold and perform stratified sampling on the training set
for train_index, test_index in skf.split(X,y):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Use stratified sampling to select the desired number of samples from each class
    X_sampled, y_sampled = resample(X_train, y_train, n_samples=sample_size, stratify=y_train, random_state=42)

    # Fit SVM classifier on the reduced dataset and evaluate performance on the test set
    svm.fit(X_sampled, y_sampled)
    y_pred = svm.predict(X_test)
    y_preds.extend(y_pred)
    y_true.extend(y_test)
    scores.append(svm.score(X_test,y_test))

# Print the average classification accuracy over all folds
print("Accuracy- SVM: {:.2f}".format(sum(scores)/len(scores)))



In [ ]:
# Assuming the true and predicted labels are stored in y_true and y_pred respectively
cm = confusion_matrix(y_true, y_preds)
np.set_printoptions(precision=3, suppress=True)
# Create a heatmap of the confusion matrix using Seaborn
sp.heatmap(cm, annot=True, cmap="Blues",fmt="d")

# Add axis labels and a title
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix for SVM ")

# Show the plot
plt.show()

In [ ]:
# y_true: true labels, y_pred_prob: predicted probabilities
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# plot ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')  # plot random curve
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for SVM')
plt.show()

## ENSEMBLE

### Voting Classifier

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# Set up KFold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize empty lists to store scores and predictions
scores = []
y_preds = []
y_tests = []

# Train the models and ensemble them
rf = RandomForestClassifier(n_estimators=100)
dt = DecisionTreeClassifier(random_state=42)
lr=LogisticRegression(C=1)
ensemble = VotingClassifier(estimators=[('rf', rf), ('dt', dt), ('lr', lr)], voting='hard')

# Loop over the splits of the data and train/test the models
for train_index, test_index in kf.split(X):
    # Split the data into training and testing sets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the models to the training data for this fold
    rf.fit(X_train, y_train)
    dt.fit(X_train, y_train)
    lr.fit(X_train, y_train)

    # Fit the ensemble model to the training data for this fold
    ensemble.fit(X_train, y_train)

    # Evaluate the performance of the ensemble model on the test set for this fold
    y_pred = ensemble.predict(X_test)
    y_preds.extend(y_pred)
    y_tests.extend(y_test)
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

# Calculate the mean and standard deviation of the cross-validation scores
mean_score = sum(scores) / len(scores)
std_dev = np.std(scores)

# Print the results
print("Ensemble accuracy: %0.4f (+/- %0.2f)" % (mean_score, std_dev * 2))


### ADA BOOST

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,VotingClassifier
from sklearn.tree import DecisionTreeClassifier

# Set up KFold cross-validation
kf = KFold(n_splits=4, shuffle=True, random_state=42)

# Initialize empty lists to store scores and predictions
scores = []
y_preds = []
y_tests = []

# Train the models and ensemble them
rf = RandomForestClassifier(n_estimators=100)
dt = DecisionTreeClassifier(random_state=42)
ada = AdaBoostClassifier(estimator=dt, n_estimators=100)

# Loop over the splits of the data and train/test the models
for train_index, test_index in kf.split(X):
    # Split the data into training and testing sets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the models to the training data for this fold
    rf.fit(X_train, y_train)
    dt.fit(X_train, y_train)
    ada.fit(X_train, y_train)

    # Combine the models into a voting classifier
    ensemble = VotingClassifier(estimators=[('rf', rf), ('ada', ada)], voting='hard')
    
    # Fit the ensemble model to the training data for this fold
    ensemble.fit(X_train, y_train)

    # Evaluate the performance of the ensemble model on the test set for this fold
    y_pred = ensemble.predict(X_test)
    y_preds.extend(y_pred)
    y_tests.extend(y_test)
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

# Calculate the mean and standard deviation of the cross-validation scores
mean_score = sum(scores) / len(scores)
std_dev = np.std(scores)

# Print the results
print("AdaBoost Ensemble accuracy: %0.4f (+/- %0.2f)" % (mean_score, std_dev * 2))